In [1]:
import sys
import os
sys.path.append('../')
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd

from utils.parser import *
from utils.helper import *
from models.data_process import *
from models.run_MLP import MLP
from anchor import anchor_tabular

args = parse_args()
device = args.device

/Users/heavenmei/miniconda3/envs/anchor/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MLP().to(device)
ckpt = torch.load('../'+args.model_path+f'MPL_{args.epoch}.pth', map_location='cpu')
model.load_state_dict(ckpt)
model.eval()

MLP(
  (net): Sequential(
    (0): Linear(in_features=102, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=2, bias=True)
  )
)

## Encoder

In [29]:
x_dataset,target,encoder = load_adult_income_dataset()
print(x_dataset.shape)

train_dataset, test_dataset, y_train, y_test = train_test_split(x_dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)

sample_id = 5
x_test = test_dataset[sample_id]
print(x_test)

(32561, 11)
[30.  9. 35.  3. 11.  4.  3.  3.  2.  0. 22.]


In [31]:
predict_fn = lambda x: model.predict_anchor(x,encoder)

print(adult_process_names)
explainer = anchor_tabular.AnchorTabularExplainer(adult_target_value, adult_process_names, train_dataset, categorical_names = {})
exp = explainer.explain_instance(x_test,predict_fn, threshold=0.95)
print(f'Anchor sample_id {sample_id}: ' ,exp.features())
print(f'Anchor sample_id {sample_id}:\n',exp.names())

['age', 'educational-num', 'hours-per-week', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
Anchor sample_id 5:  [5, 1]
Anchor sample_id 5:
 ['marital-status > 2.00', 'educational-num <= 9.00']


In [ ]:
# exp.show_in_notebook()